# Дескрипторы

https://habr.com/ru/post/122082/

Чтобы понять, что такое дескриптор, обратимся к документации питона:

In general, a descriptor is an object attribute with “binding behavior”, one whose attribute access has been overridden by methods in the descriptor protocol. Those methods are `__get__()`, `__set__()`, and `__delete__()`. If any of those methods are defined for an object, it is said to be a descriptor.

### Что это означает

Дескриптор - это любой объект, который "привязывается" к объекту класса в виде атрибута это класса, при условии, если в нём определен способ этого привязывания - методы `__get__()`, `__set__()` и `__delete__()`.

### Когда это происходит

Эти методы вызываются тогда, когда мы обращаемся к дескриптору через объект класса и точку: `obj.our_descriptor_attr`.

### Нюансы

- Если определен один из перечисленных методов - объект считается дескриптором;
- если объект дескриптора определяет `__get__`, `__set__` - data дескриптором;
- если объект дескриптора определяет `__get__` - non-data дескриптором.

Они отличаются приоритетом вызова по отношению к полю `__dict__`.

## Data-дескриптор

In [4]:
class DataDescriptor:
    def __get__(self, owner, owner_cls):
        print("__get__")
        print(f"Вызов из объекта: {owner}")
        print(f"Класс вызывающего объекта: {owner_cls}")
        print()

    def __set__(self, owner, val):
        print("__set__")
        print(f"Присвоение значения {val} объекту {owner}")
        print()

    def __delete__(self, owner):
        print("__del__")
        print(f"Удаляем атрибут из объекта {owner}")
        print()


class SomeData:
    data = DataDescriptor()


d = SomeData()
SomeData.data  # вот тут будет вызван __get__ с owner == None
d.data
d.data = 1
del d.data

__get__
Вызов из объекта: None
Класс вызывающего объекта: <class '__main__.SomeData'>

__get__
Вызов из объекта: <__main__.SomeData object at 0x0000022483244460>
Класс вызывающего объекта: <class '__main__.SomeData'>

__set__
Присвоение значения 1 объекту <__main__.SomeData object at 0x0000022483244460>

__del__
Удаляем атрибут из объекта <__main__.SomeData object at 0x0000022483244460>



In [2]:
d.data

__get__
Вызов из объекта: <__main__.SomeData object at 0x0000022483229FA0>
Класс вызывающего объекта: <class '__main__.SomeData'>



Видим, что при обращении к дескриптору срабатывают его магические методы. Однако стоит понимать, что они вызываются только тогда, когда обращение к атрибуту происходит через точку. Например, в следующем примере setter не будет вызван:

In [7]:
d.__dict__

{'data': 1}

In [6]:
d.__dict__['data'] = 1
d.data

__get__
Вызов из объекта: <__main__.SomeData object at 0x0000022483244460>
Класс вызывающего объекта: <class '__main__.SomeData'>



In [ ]:
d.__dict__

In [8]:
d.__getattribute__("data")

__get__
Вызов из объекта: <__main__.SomeData object at 0x0000022483244460>
Класс вызывающего объекта: <class '__main__.SomeData'>



Поскольку дескриптор - это поле класса, то присвоение в это поле любого другого объекта просто удалит ссылку на дескриптор:

In [9]:
SomeData.data = 1
print(SomeData.data)
print(d.data)
print(SomeData().data)
del SomeData.data
print(SomeData.data)

1
1
1


AttributeError: type object 'SomeData' has no attribute 'data'

## Non-data дескриптор

In [10]:
class NonDataDescriptor:
    def __get__(self, obj, cls):
        print(f"Вызов из объекта: {obj}")
        print(f"Класс вызывающего объекта: {cls}")
        print()


class SomeData:
    data = NonDataDescriptor()


d = SomeData()
SomeData.data  # вот тут будет вызван __get__ с obj None
d.data

print("А теперь изменим d.data. Метод __set__ дескриптора не определен, поэтому ссылка в переменной d.data перезаписалась на 1:")
d.data = 1
print(d.data)

Вызов из объекта: None
Класс вызывающего объекта: <class '__main__.SomeData'>

Вызов из объекта: <__main__.SomeData object at 0x0000022483244790>
Класс вызывающего объекта: <class '__main__.SomeData'>

А теперь изменим d.data. Метод __set__ дескриптора не определен, поэтому ссылка в переменной d.data перезаписалась на 1:
1


## Резюме

- дескрипторы вызываются с помощью метода `__getattribute__`
- переопределение `__getattribute__` прекратит автоматический вызов дескрипторов
- `__getattribute__` доступен только внутри классов и объектов нового стиля
- `object.__getattribute__` и `type.__getattribute__` делают разные вызовы к `__get__`
- дескрипторы данных всегда имеют преимущество перед переменными объекта
- дескрипторы не данных могут потерять преимущество из-за переменных объекта

## Еще примеры
### Дескриптор данных с проверкой значения

In [11]:
class NonNegative:
    def __set_name__(self, owner, name):
        self.name = name
        
    def __set__(self, owner, value):
        error_msg = "Значение должно быть целым неотрицательным числом"
        if not isinstance(value, int):
            raise TypeError(error_msg)
        if value < 0:
            raise ValueError(error_msg)
        owner.__dict__[self.name] = value
        print(f"setter: {self.name}")
        
    def __get__(self, owner, owner_cls):
        print(f"getter: ")
        return owner.__dict__[self.name]
    
    
class Human:
    height = NonNegative()
    weight = NonNegative()
    age = NonNegative()
    
    def __init__(self, name, age, height, weight):
        self.name = name
        self.age = age
        self.height = height
        self.weight = weight

In [14]:
ivan = Human("Ivan", 25, 180, 80)

setter: age
setter: height
setter: weight


In [15]:
ivan.height

getter: 


180

In [16]:
ivan.__dict__

{'name': 'Ivan', 'age': 25, 'height': 180, 'weight': 80}

## Зачем нужны дескрипторы

В питоне многие вещи написаны с использованием дескрипторов, например, property. В общем случае это гибкий инструмент для работы с атрибутами класса со стороны самого атрибута, а встроенные решения предоставляют более высокоуровневый SDK к дескрипторам. Рассмотрим реализацию property:

In [17]:
class C:
    def getx(self): return self._x

    def setx(self, value): self._x = value

    def delx(self): del self._x

    x = property(getx, setx, delx, "I'm the 'x' property.")


class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)


class CWithDescriptor:
    def getx(self): return self._x

    def setx(self, value): self._x = value

    def delx(self): del self._x

    x = Property(getx, setx, delx, "I'm the 'x' property.")


c = C()
c1 = CWithDescriptor()
c.x = 1
c1.x = 1
print('c.__dict__', c.__dict__)
print('c1.__dict__', c1.__dict__)

c.__dict__ {'_x': 1}
c1.__dict__ {'_x': 1}


Staticmethod и classmethod - тоже дескрипторы.

In [18]:
class E:
    def f(x):
        print(x)

    f = staticmethod(f)


class StaticMethod(object):
    "Emulate PyStaticMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, objtype=None):
        return self.f


class Analog:
    @StaticMethod
    def p():
        print("hi")
        
        
Analog.p()

hi


In [19]:
class ClassMethod(object):
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, klass=None):
        if klass is None:
            klass = type(obj)

        def newfunc(*args):
            return self.f(klass, *args)

        return newfunc
    
    
class Analog:
    @ClassMethod
    def p(cls):
        print("class is:", cls)
        
        
Analog.p()

class is: <class '__main__.Analog'>


## Привязка методов

Кстати говоря, если нам нужно вызывать один и тот же метод много раз, то быстрее будет его отвязать от объекта:

In [20]:
class A:
    def foo(self):
        pass


a = A()
binding = a.foo

In [21]:
%%timeit

a.foo()

72 ns ± 2.93 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [22]:
%%timeit

binding()

68.9 ns ± 3.89 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Задание

Написать дескриптор, который будет хранить значение, но по вызову метода `null` дескриптора все значения всех инициализированных дескрипторов обнулятся.

Подсказка: для этого нужно хранить внутри дескриптора ссылки на все содержащие его классы и список названий атрибутов, в которых записана ссылка на дескриптор (названия можно получить, например, через `__dict__` объекта, сравнив класс атрибута с классом-дескриптором).